숙대 학과 정보 수집

In [9]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
import os

In [10]:
# 접속객체 생성 
session = requests.Session() 
# 접속 객체에 header 부가정보 삽입
session.headers.update({
    "Referer" : "",
    "User-Agent" :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36})"
})

In [11]:
# 대학번호 리스트 정의 --> URL을 분석해서 얻은 값
# 대학별로 url 주소가 path파라미터라서, 중간 주소를 가져옴 
clist = [1011,1023,1030,1041,1046,1052,1054,1057,1062,1064,1070,1073,1076]

응답결과로부터 데이터 추출하기

In [12]:
# 수집할 정보를 저장할 빈 리스트
학과목록_리스트 = []

# 대학 수 만큼 반복한다. 
for c in clist : 

    # 접속 주소 (대학별로 돌면서 취합한다.)
    url = 'https://www.sookmyung.ac.kr/sookmyungkr/%d/subview.do' % c

    # 접속 객체를 활용해 API에 접속
    r = session.get(url) # r은 soup에서 사용되었기 때문에 이 변수명을 다시 사용할 수 있다. 

    # 실패 시
    if r.status_code != 200 : # 성공이 아니라며
        msg = "[%d Error]%s 에러가 발생함" %(r.status_code,r.reason)
        raise Exception(msg) # 에러를 강제로 발생

    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text) # 객체 생성 <- 이제 soup를 사용하면 안되는.... 


    college_list = soup.select('.college_list')
    #print(college_list)
        
    for item in college_list : # html에서 가져오기 item 학과 하나씩임 
        # print(item)
        # print('-'*50)
        
        # 학과이름
        h5El = item.select("h5")
        name = h5El[0].text.strip()
        #print(name)

        # 가져온 페이지 주소는 /sookmyungkr/1012/subview.do 라서 앞부분 https://~~~ac.kr을 붙여줘야 한다.
        # 같은 사이트에서 주소를 걸 때는 현재주소/sook.. 이렇게 시작해도 되지만 외부에서 (파이썬에서) 부를때는 전체 주소를 넣어줘야 한다. 
        # 상세보기 URL
        viewEl = item.select('.view')
        view = viewEl[0].attrs['href'] # '상세보기'에 들어가서 모두 가져와야 하니까 a href 

        # 상세보기 url full domain     
        # find는 인덱스를 리턴하는데 없으면 -1을 반환함 -> 예외처리로 사용할 수 있다. 
        if view.find('https://www.sookmyung.ac.kr') == -1 : #찾았는데 없으면 -1 반환
            view = 'https://www.sookmyung.ac.kr' + view
        # print(view)

        # 학과소개 pdf 
        pdfEl = item.select('.info')
        pdf = pdfEl[0].attrs['href']
        # print(info) # https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_01_2022.pdf

        # 학과별 홈페이지
        homepageEl = item.select('.homepage')
        homepage = homepageEl[0].attrs['href']
        # print(homepage) # https://korean.sookmyung.ac.kr/

        # 수집한 값들을 딕셔너리로 묶는다. 
        college_dict = {'학과명':name,'상세페이지':view,'홈페이지':homepage}




        # 상세보기 url을 새롭게 수집
        r = session.get(view) # 접속 객체를 활용해 API에 접속


        # 태그 구조는 같지만 클래스가 없으면 위의 클래스로 찾아가서 자손 -> 컬리지 -> dd 
        if r.status_code != 200 :
            msg = "[%d Error]%s 에러가 발생함" %(r.status_code,r.reason)
            #print(msg)
            continue

        r.encoding = 'utf-8'
        detailSoup = BeautifulSoup(r.text)
        # print(detailSoup)

        # 같은 구조를 갖는 전화번호, 팩스번호, 위치, 이메일 주소를 가져온다.

        infoEl = detailSoup.select("college_info_data dl")
        # print(infoEl)
        # print("------"*30)

# f12 -> 클래스 -> 요소 선택하면 다 복사됨 
# 최대한 범용적인 조건으로 수집해야 한다. 

        for info in infoEl :
            print(info)
            dt = info.select("dt")[0].text.replace(':',"").strip()
            dd = info.select("dd")[0].text.replace(':',"").strip() # 자손은 띄어쓰기로도 구분됨 

            college_dict[dd]
    

        학과목록_리스트.append(college_dict)

        if pdf.find(".pdf") > -1 : # 만약 pdf 파일이 있다면 
            r = session.get(pdf, stream=True)
            if r.status_code == 200 :# 성공했을때만 저장해라. 에러 하나때문에 다 저장 못할 수 없으니까. 
                r.encoding = 'utf-8' 
                with open('%s.pdf' % name, 'wb') as f :
                    f.write(r.raw.read()) 


df = DataFrame(학과목록_리스트)
# df.to_excel('숙대학과목록.xlsx')
df

# path ='./'
# os.listdir(path)

['01-SimpleCralwer.ipynb.pdf',
 '01_SimpleCralwer.ipynb',
 '01_실습_html입력.html',
 '02-블로그_게시글_수집.ipynb.pdf',
 '02_블로그_게시글_수집.ipynb',
 '03-학과수집(step1).ipynb.pdf',
 '03-학과수집(step2).ipynb.pdf',
 '03-학과수집(step3).ipynb.pdf',
 '03-학과수집(step4).ipynb',
 '03-학과수집(step4).ipynb.pdf',
 'sample_html.pdf',
 'test.xlsx',
 '가족자원경영학과.pdf',
 '경영학부.pdf',
 '경제학부.pdf',
 '공예과.pdf',
 '관현악과.pdf',
 '교육학부.pdf',
 '글로벌협력전공.pdf',
 '기계시스템학부.pdf',
 '기초공학부.pdf',
 '독일언어ㆍ문화학과.pdf',
 '무용과.pdf',
 '문헌정보학과.pdf',
 '문화관광외식학부르꼬르동블루외식경영전공.pdf',
 '문화관광외식학부문화관광학전공.pdf',
 '법학부.pdf',
 '사회심리학과.pdf',
 '산업디자인과.pdf',
 '생명시스템학부.pdf',
 '성악과.pdf',
 '소비자경제학과.pdf',
 '소프트웨어학부데이터사이언스전공.pdf',
 '소프트웨어학부컴퓨터과학전공.pdf',
 '수학과.pdf',
 '숙대학과목록.xlsx',
 '시각ㆍ영상 디자인과.pdf',
 '식품영양학과.pdf',
 '아동복지학부.pdf',
 '앙트러프러너십전공.pdf',
 '약학부.pdf',
 '역사문화학과.pdf',
 '연습문제(1)-쿠팡검색결과-수집.ipynb.pdf',
 '연습문제(1)=쿠팡검색결과-수집.ipynb',
 '영어영문학전공.pdf',
 '의류학과.pdf',
 '인공지능공학부.pdf',
 '일본학과.pdf',
 '작곡과.pdf',
 '정치외교학과.pdf',
 '중어중문학부.pdf',
 '첨단소재·전자융합공학부신소재물리전공.pdf',
 '첨단소재·전자융합공학부지능형전자시스템전공